# Étude Statistique Accident d'avion - Scraping

L'analyse de la probabilité de mourir dans un accident d'avion est une étude statistique qui vise à évaluer le risque de décès lors d'un vol. Cette analyse utilise des données sur les accidents d'avion passés pour estimer la probabilité de décès en cas d'accident. Elle peut également tenir compte de facteurs tels que la compagnie aérienne, le type d'avion et les itinéraires de vol pour affiner les résultats. L'objectif de cette analyse est d'aider les voyageurs à comprendre les risques associés aux voyages en avion.

Pour se faire nous utiliserons la base de données de l'ASN qui recense tous les accidents d'avions dans le monde.
https://aviation-safety.net/database/

Combiné à la cela pour estimer la probabilité de subir un accident, nous allons utiliser les données Statistica sur le traffic aérien par année de 2004 à ce jour.

https://www.statista.com/statistics/564769/airline-industry-number-of-flights/

J'aurais apprécié une base de données plus détaillée avec le trafic par pays mais il s'agit du mieux que j'ai pu trouver en open data.

In [294]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd

from selenium.webdriver.common.keys import Keys

import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [288]:
years = list(range(1960, 2023))
urls = [f"https://aviation-safety.net/database/dblist.php?Year={year}" for year in years]


In [303]:
driver = webdriver.Chrome()

In [302]:
#Les listes qui seront utilisés pour générer les colonnes du dataframe
date_list = []
model_list = []
operator_list = []
fat_list = []
loc_list = []
cat_list = []
register_list = []

In [291]:
print(len(urls))

63


In [ ]:

for i,url in enumerate(urls):
    driver.get(url)
    
    page_no = 2
    
    #Tant qu'il existe une table et une pagination, le script continue à scraper
    while True:
        try:
            table = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "table")))
            rows = table.find_elements(By.TAG_NAME, "tr")
            
            #Récupère tous les éléments HTML "<td>" qui se trouvent dans chaque ligne "<tr>"
            #Autrement dit toutes les colonnes pour chaque lignes
            for row in rows:
                cells = row.find_elements(By.CSS_SELECTOR, "td")

                #Vérifie si le nombre de cellules est égal à 9. 
                #Cela permet d'exclure les en-têtes de table qui ne contiennent pas d'informations de crash.
                
                if len(cells) == 9 :
                    date0 = cells[0].text.strip()
                    date_list.append(date0)

                    model0 = cells[1].text.strip()
                    if model0 == '' :
                        model0 = None
                    model_list.append(model0)

                    register0 = cells[2].text.strip()
                    if register0 == '' :
                        register0 = None
                    register_list.append(register0)

                    operator0 = cells[3].text.strip()
                    if operator0 == '' :
                        operator0 = None
                    operator_list.append(operator0)

                    fat0 = cells[4].text.strip()
                    if fat0 == '' :
                        fat0 = None
                    fat_list.append(fat0)

                    cat0 = cells[8].text.strip()
                    if cat0 == '' :
                        cat0 = None
                    cat_list.append(cat0)

                    img = cells[6].find_element(By.XPATH,'.//img')
                    loc0 = img.get_attribute('title')
                    if loc0 == '' :
                        loc0 = None
                    loc_list.append(loc0)

            try:
                page_no += 1
                print("Trying :",page_no)
                
                #Tente d'aller à la page suivante
                driver.find_element(By.CSS_SELECTOR, f'a[href*="page={page_no}"]').click()
                print(len(date_list))
            except:
                print("No more page")
                break

        except StaleElementReferenceException:
            print("Stale element reference. Trying again...")
            continue

    print("Avancement:", i/len(urls)*100)


In [305]:
data_dict = {
    'Date': date_list,
    'Model': model_list,
    'Operator': operator_list,
    'Fatalities': fat_list,
    'Location': loc_list,
    'Category': cat_list,
    'Registration': register_list
}

# create a DataFrame from the dictionary
df = pd.DataFrame(data_dict)
df.head()

Date                          Model           Operator Fatalities  \
0  03-JAN-1960           Douglas C-47A (DC-3)    Indian Airlines          9   
1  03-JAN-1960  Lockheed L-749A Constellation  Eastern Air Lines          0   
2  04-JAN-1960                  Curtiss C-46A      T.A. Salvador          0   
3  04-JAN-1960             U-1A Otter (DHC-3)            US Army         10   
4  05-JAN-1960           Vickers 701 Viscount                BEA          0   

  Location Category Registration  
0    India       A1       VT-CGG  
1      USA       A1        N110A  
2   Brazil       A1       PP-SLJ  
3    Libya       A1      55-2974  
4    Malta       A1       G-AMNY

In [306]:
df.shape

(8934, 7)

In [308]:
#Export des résultats 
df.to_csv("ASN Database.csv",index=False)